In [3]:
import praw

In [2]:
import requests
import pandas as pd
import numpy as np
from tqdm import tqdm

In [9]:
def reddit_scrape (subreddit = 'Singapore', utc = 1647150177,
                   url = 'https://api.pushshift.io/reddit/search/submission',
                   number = 100000):
    #setting parameters to extract 1st 100 posts
    params = {'subreddit': subreddit,
              'size':100, #max number of posts for pullshift api
              'before': utc}
    res = requests.get(url,params)
    data = res.json()
    posts = data['data']
    # creating a dataframe from the posts scrapped
    df = pd.DataFrame(posts)
    # creating a loop where as long as the dataframe does not have 5000 posts we will keep pulling 100 posts or the number required to hit 5000 posts, whichever is less
    while len(df)< number:
        try:
            remainder = number-len(df)
            # setting the size to either 100 posts or the number required to hit 5k posts
            size = np.min((remainder, 100))
            # parameters for requests, similar to the initial parameters however cut off date will be based on the date of the last item in the current batch of posts.
            params_for_additions = {'subreddit': subreddit,
                                    'size': size,
                                    'before': df.created_utc.iloc[-1]}

            additional_requests = requests.get(url, params_for_additions)
            additional_data = additional_requests.json()
            additional_posts = additional_data['data']
            #Adding newly extracted posts to the created df
            df = pd.concat([df, pd.DataFrame(additional_posts)], axis=0)
            #dropping any duplicates
            df.drop_duplicates(subset = ['title'], keep = 'last', inplace = True)
            #resetting index
            df.reset_index(inplace = True, drop = True)
    #Once X-num unique posts have been created, save data into a csv
        except:
            pass
    df.to_csv(f'{subreddit}.csv')
    return(df)

In [10]:
%%time
all_subs = reddit_scrape()

CPU times: user 5min 41s, sys: 46.1 s, total: 6min 27s
Wall time: 2h 22s


In [5]:
all_subs = pd.read_csv('submission_info.csv')

C:\Users\Tye_H\AppData\Local\Temp/ipykernel_8704/22619832.py:1: DtypeWarning: Columns (2,4,12,14,15,23,31,65,68,71,72,74,83,84,86,87,89,91,92,93,95,96) have mixed types. Specify dtype option on import or set low_memory=False.
  all_subs = pd.read_csv('100k_subs.csv')


In [6]:
all_subs.to_csv('submission_info.csv')

In [7]:
#reddit = praw.Reddit(client_id = 'reddit clied id',
                     #client_secret = 'reddit client secret',
                     #user_agent = 'reddit username')

In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
comments_all = pd.DataFrame(columns=['user', 'comment', 'date_time', 'sub_title'])

In [9]:
%%time
for id in tqdm(all_subs.id[50000:]):
    post = reddit.submission(id = id)
    post.comments.replace_more(limit = None)
    for comments in post.comments.list():
        comments_all = comments_all.append({'user' : comments.author,
                                            'comment' : comments.body,
                                            'date_time': comments.created_utc,
                                            'sub_title': post.title},
                                           ignore_index = True)

  3%|▎         | 1530/50000 [2:46:50<88:05:44,  6.54s/it]   


NotFound: received 404 HTTP response

In [10]:
comments_all.to_csv('submissions_comments.csv') # full file isnt available on Github due to size limits

In [11]:
comments_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47201 entries, 0 to 47200
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user       41014 non-null  object
 1   comment    47201 non-null  object
 2   date_time  47201 non-null  object
 3   sub_title  47201 non-null  object
dtypes: object(4)
memory usage: 1.4+ MB
